## Barrier Option Pricing
It is an option whose payoff is conditional upon the underlying asset's price breaching a barrier level during the option's lifetime. the payoff depends on whether or not the underlying asset has reached/exceeded a predetermined price threshold. Barrier options are typically classified as one of the following:

• Knock-out: option becomes worthless if the underlying asset’s price exceeds a certain threshold

• Knock-in: option has no value until the underlying asset’s price reaches a certain threshold

### Geometric Brownian Motion (GBM) Formula:

GBM model for the price S_t of an asset at time  t  is given by:


$$S_t = S_0 \exp \left( \left( \mu - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right)$$


### Final Price Formula:


$$ S_t = S_0 \cdot \exp \left( \left( \mu - \frac{1}{2} \sigma^2 \right) \cdot \text{time\_steps} + \sigma \cdot W_t \right) $$

where : 

	•	 S_0 : initial price at t = 0
	•	 mu : drift (mean return) 
	•	 sigma : volatility 
	•	 W_t : Wiener process (also known as Brownian motion) at time  t 
	•	 t : time (scaled appropriately)



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [5]:
def simulate_gbm(s_0, mu, sigma, n_sims, T, N, random_seed=None):
    np.random.seed(random_seed)  # Set random seed for reproducibility
    dt = T / N  # Time step size
    time_steps = np.linspace(0, T, N)  # Time grid (length N)

    # Create a 2D array of random variables (Wiener process increments)
    W = np.random.normal(size=(n_sims, N)) * np.sqrt(dt)
    
    # Cumulative sum of random variables to get the Wiener process
    W = np.cumsum(W, axis=1)

    # Create a 2D array for time_steps, where each row is the same time grid
    time_steps_2d = np.broadcast_to(time_steps, (n_sims, N))

    # Ensure mu and sigma are scalars
    mu = float(mu)  # Convert to scalar
    sigma = float(sigma)  # Convert to scalar

    # Debugging: Print shapes and types
    print(f"mu: {mu}, type: {type(mu)}")
    print(f"sigma: {sigma}, type: {type(sigma)}")
    print(f"time_steps_2d shape: {time_steps_2d.shape}")
    print(f"W shape: {W.shape}")

    # Now we calculate S_t for each simulation path
    S_t = s_0 * np.exp((mu - 0.5 * sigma**2) * time_steps_2d + sigma * W)

    # Insert the initial price at time t=0 for each simulation
    S_t = np.insert(S_t, 0, s_0, axis=1)

    return S_t

In [41]:
S_0 = 65
K = 78
BARRIER = 65
r = 0.06
sigma = 0.2
T =1
N = 252
dt = T / N
N_SIMS = 10 ** 5
OPTION_TYPE = "call" 
discount_factor = np.exp(-r * T)

In [43]:
gbm_sims = simulate_gbm(s_0=S_0, mu=r, sigma=sigma,
                        n_sims=N_SIMS, T=T, N=N)

mu: 0.06, type: <class 'float'>
sigma: 0.2, type: <class 'float'>
time_steps_2d shape: (100000, 252)
W shape: (100000, 252)


In [45]:

max_value_per_path = np.max(gbm_sims, axis=1)

In [47]:
payoff = np.where(max_value_per_path > BARRIER,
                  np.maximum(0, gbm_sims[:, -1] - K),
0)

In [49]:
premium = discount_factor * np.mean(payoff)
premium

2.288155718827228